# Dataset Deduplication for `java-pretrain`
To have as much data as possible for our pretraing experiments we combined the training partitions of code2seq's `java-small`, `java-medium` and `java-large` datasets.  
However, these datasets are not completely separated and thus, the training partition of `java-large` can potentially contain Java methods from the test partition of `java-small`. To alleviate this issue, we perform a strict deduplication between the new training partition of `java-pretrain` and the valid/test partitions of `java-small` and `java-medium`.  
For this, we performed the following steps:
 1. Copy the training partitions of `java-small`, `java-medium` and `java-large` into a new `java-pretrain/training` folder. The valid and test partition for this new `java-pretrain` dataset is not important for our use-case as we only use it for self-supervised language model pretraining.
 2. Perform harsh project-level deduplication by running this notebook until step 5
 3. Compute more granular file-level similarities by running the `deduplicate-java-pretrain.py` script for the datasets and partitions you wish to deduplicate against. In our case, we deduplicated against the valid and train partitions of `java-small` and `java-medium`. This will store a pickled list of files to delete in `java-pretrain`.
 4. Load this list in step 5 of this notebook to inspect what is to be deleted and finally run the file-level deletion

In [ ]:
%cd ../..
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import re
import shutil
from collections import defaultdict
from difflib import SequenceMatcher
from pathlib import Path

from tqdm import tqdm

from code_transformer.utils.io import save_pickled, load_pickled
from env import CODE2SEQ_RAW_DATA_PATH

# 1 Data Setup

In [ ]:
data_path_code2seq = CODE2SEQ_RAW_DATA_PATH

In [ ]:
dataset = 'java-small'
partition = 'test'
projects_folder = Path(f"{data_path_code2seq}/{dataset}/{partition}")

In [ ]:
reference_dataset = 'java-pretrain'
reference_partition = 'training'
reference_projects_folder = Path(f"{data_path_code2seq}/{reference_dataset}/{reference_partition}")
reference_projects = {p for p in reference_projects_folder.iterdir()}

# 2 Helper Methods

In [ ]:
def get_files_recursive(folder):
    if not folder.is_dir():
        return [folder]
    results = []
    for file in folder.iterdir():
        if file.is_dir():
            results.extend(get_files_recursive(file))
        else:
            results.append(file)
    return results

In [ ]:
def recursive_search(folder, file_name):
    print(folder)
    results = []
    for file in folder.iterdir():
        if file.is_dir():
            results.extend(recursive_search(file, file_name))
        elif file.stem == file_name:
            results.append(file)
    return results

In [ ]:
def remove_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    # first group captures quoted strings (double or single)
    # second group captures comments (//single-line or /* multi-line */)
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    def _replacer(match):
        # if the 2nd group (capturing comments) is not None,
        # it means we have captured a non-quoted (real) comment string.
        if match.group(2) is not None:
            return "" # so we will return empty to remove the comment
        else: # otherwise, we will return the 1st group
            return match.group(1) # captured quoted-string
    return regex.sub(_replacer, string)

In [ ]:
def read_file(f):
    try:
        return f.read_text()
    except UnicodeDecodeError:
        return f.read_text(encoding='cp1252')

In [ ]:
def similar(a, b):
    return SequenceMatcher(None, a, remove_comments(read_file(b))).ratio()

In [ ]:
def delete_project_files(projects_to_delete):
    for p in projects_to_delete:
        path = Path(f"{reference_projects_folder}/{p}")
        if path.exists():
            print(path)
            shutil.rmtree(path)

# 3 File Lookup
The underlying assumption is that if a code file is duplicated then its clone will have the same file name. Thus, we create a mapping from file name => path for a quick lookup of potential duplication candidates

In [ ]:
file_lookup = defaultdict(list)
for p in tqdm(list(reference_projects_folder.iterdir())):
    for f in get_files_recursive(p):
        file_lookup[f.stem].append(f)

In [ ]:
save_pickled(file_lookup, "file_lookup")

# 4 Duplication detection
 1. Iterate through all projects in java-small/medium  
 2. If a project has the exact same name as one in java-pretrain, we instantly mark the whole project folder to be deleted
 3. Get all files within this project  
 4. For every file:  
     - find duplication candidates by consulting lookup  
     - Restrict set of candidate duplicates to files where the file sizes of search and candidate file differ by at most 5%  
 5. Detect projects in java-pretrain that have a large overlap with the project, i.e., > 25% of the files in the search project have candidates in the candidate project  
 6. These projects will later be deleted from java-pretrain

In [ ]:
results = dict()
projects_to_delete = []
for p1 in tqdm(list(projects_folder.iterdir())):
    search_files = get_files_recursive(p1)
    num_files = len(search_files)
    print(p1.stem, num_files)

    project_candidates = defaultdict(list)
    if p1.stem in {p.stem for p in reference_projects}:
        projects_to_delete.append(p1.stem)

    for i, search_file in enumerate(search_files):
        print(f"{i}/{num_files}", end='\r')
        similar_files = []
        for candidate_file in file_lookup[search_file.stem]: 
            size1 = os.path.getsize(search_file)
            size2 = os.path.getsize(candidate_file)
            if size2 == 0:
                size_diff = 1 if size1 == size2 else 0
            else:
                size_diff = size1 / size2
            if size_diff > 0.95 and size_diff < 1.05:
                similar_files.append(candidate_file)
                project_candidates[candidate_file.parts[len(reference_projects_folder.parts)]].append(candidate_file)


    for k in project_candidates.keys():
        project_candidates[k] = len(set(project_candidates[k]))
    results[p1.stem] = (project_candidates, num_files)

In [ ]:
print(projects_to_delete)

In [ ]:
delete_project_files(projects_to_delete)

## Find project candidates in java-pretrain

In [ ]:
projects_to_delete_2 = set()
for project_name, (project_candidates, num_files) in results.items():
    for project_candidate, num_matches in project_candidates.items():
        if num_matches / num_files > 0.25 and num_files > 100:
            projects_to_delete_2.add(project_candidate)
            print(project_name, project_candidate)

In [ ]:
print(projects_to_delete_2)

In [ ]:
delete_project_files(projects_to_delete_2)

# 5 File similarity deduplication
The above steps are only a course filtering step to already delete obvious duplicates. It can still be that files from different projects are very similar. This can only be detected by comparing the contents. This is an expensive computation and is done by the `deduplicate-java-pretrain.py` script. The result of this script are lists of files that should be deleted. Deletion is done manually here.

In [ ]:
dataset = 'java-small'
partition = 'test'

In [ ]:
files_to_delete = load_pickled(f"{CODE2SEQ_RAW_DATA_PATH}/java-pretrain/files_to_delete_{dataset}_{partition}.p")

In [ ]:
for f in tqdm(files_to_delete):
    if f.exists():
        f.unlink()